# Media Library

> Unified facade for media file discovery, mounting, and access

In [ ]:
#| default_exp media.library

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import List, Optional
from fasthtml.common import APIRouter
from fastcore.basics import patch

from cjm_fasthtml_workflow_transcription_single_file.media.config import MediaConfig
from cjm_fasthtml_workflow_transcription_single_file.media.models import MediaFile
from cjm_fasthtml_workflow_transcription_single_file.media.scanner import MediaScanner
from cjm_fasthtml_workflow_transcription_single_file.media.mounter import MediaMounter
from cjm_fasthtml_workflow_transcription_single_file.media.pagination import create_media_pagination
from cjm_fasthtml_workflow_transcription_single_file.media.file_selection_pagination import create_file_selection_pagination as create_fs_pagination

## MediaLibrary Class

A unified facade that combines `MediaScanner` and `MediaMounter` functionality. Provides a simple interface for browsing media files and accessing their URLs.

In [ ]:
#| export
class MediaLibrary:
    """Unified interface for media scanning, mounting, and browsing."""

    def __init__(
        self,
        config: MediaConfig  # Media configuration with directories and settings
    ):
        """Initialize the media library."""
        self.config = config
        self.scanner = MediaScanner(config)
        self.mounter = MediaMounter()
        self._pagination = None
        self._pagination_router = None

    @property
    def pagination(self):
        """Access to the pagination instance."""
        return self._pagination

    @property
    def pagination_router(self) -> Optional[APIRouter]:
        """Access to the pagination router."""
        return self._pagination_router

    @property
    def file_selection_pagination(self):
        """Access to the file selection pagination instance."""
        return getattr(self, '_file_selection_pagination', None)

    @property
    def file_selection_router(self) -> Optional[APIRouter]:
        """Access to the file selection pagination router."""
        return getattr(self, '_file_selection_router', None)

In [ ]:
#| export
@patch
def mount(
    self: MediaLibrary,
    app  # FastHTML/Starlette application instance
) -> None:
    """Mount media directories to app for static file serving."""
    self.mounter.mount(app, self.config.directories)

In [ ]:
#| export
@patch
def scan(
    self: MediaLibrary,
    force_refresh: bool = False  # Force a fresh scan, ignoring cache
) -> List[MediaFile]:  # List of MediaFile objects
    """Scan for media files."""
    return self.scanner.scan(force_refresh)

In [ ]:
#| export
@patch
def get_transcribable_files(
    self: MediaLibrary
) -> List[MediaFile]:  # List of MediaFile objects with media_type 'audio' or 'video'
    """Get files suitable for transcription (audio and video only)."""
    files = self.scan()
    return [f for f in files if f.media_type in ["audio", "video"]]

In [ ]:
#| export
@patch
def get_url(
    self: MediaLibrary,
    file_path: str  # Full path to the media file
) -> Optional[str]:  # URL path to access the file, or None if not in a mounted directory
    """Get URL for a media file."""
    return self.mounter.get_url(file_path)

In [ ]:
#| export
@patch
def clear_cache(
    self: MediaLibrary
) -> None:
    """Clear the scan cache."""
    self.scanner.clear_cache()

In [ ]:
#| export
@patch
def get_summary(
    self: MediaLibrary
) -> dict:  # Dictionary with file counts, sizes, and breakdowns
    """Get summary statistics for scanned files."""
    return self.scanner.get_summary()

In [ ]:
#| export
@patch
def create_pagination(
    self: MediaLibrary,
    pagination_id: str,  # Unique identifier for this pagination instance
    content_id: str,  # HTML ID for the content area
    preview_route_func = None,  # Optional function to generate preview route URL
    modal_id: str = "sf-media-preview"  # ID for the preview modal
):  # Configured Pagination instance
    """Create a pagination instance for browsing media files."""
    self._pagination = create_media_pagination(
        pagination_id=pagination_id,
        scanner=self.scanner,
        mounter=self.mounter,
        items_per_page=self.config.items_per_page,
        content_id=content_id,
        preview_route_func=preview_route_func,
        modal_id=modal_id
    )
    return self._pagination

In [ ]:
#| export
@patch
def get_pagination_router(
    self: MediaLibrary,
    prefix: str  # URL prefix for pagination routes
) -> Optional[APIRouter]:  # APIRouter for pagination, or None if pagination not created
    """Get the pagination router for registration with the app."""
    if self._pagination:
        self._pagination_router = self._pagination.create_router(prefix=prefix)
        return self._pagination_router
    return None

In [ ]:
#| export
@patch
def create_file_selection_pagination(
    self: MediaLibrary,
    pagination_id: str,  # Unique identifier for this pagination instance
    content_id: str,  # HTML ID for the content area
    preview_url_func = None,  # Function that takes file index and returns preview URL
    preview_target_id: str = None  # HTML ID to target for preview modal
):  # Configured Pagination instance for file selection
    """Create a pagination instance for file selection table with radio buttons."""
    self._file_selection_pagination = create_fs_pagination(
        pagination_id=pagination_id,
        scanner=self.scanner,
        items_per_page=self.config.items_per_page,
        content_id=content_id,
        preview_url_func=preview_url_func,
        preview_target_id=preview_target_id
    )
    return self._file_selection_pagination

In [ ]:
#| export
@patch
def get_file_selection_router(
    self: MediaLibrary,
    prefix: str  # URL prefix for pagination routes
) -> Optional[APIRouter]:  # APIRouter for file selection pagination, or None if not created
    """Get the file selection pagination router."""
    if self._file_selection_pagination:
        self._file_selection_router = self._file_selection_pagination.create_router(prefix=prefix)
        return self._file_selection_router
    return None

## Usage Examples

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()